NODELTA WALA HAI YE

In [1]:
import tensorflow as tf
import os
import numpy as np
import pickle
from tensorflow.keras.layers import Activation
#os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [2]:
### MODEL ###

In [3]:
def activation_fn(x):
    value = (5*(tf.math.exp(x/5) - tf.math.exp(-x/5))/(tf.math.exp(x/5) + tf.math.exp(-x/5)))
    return value

In [4]:
from keras.utils.generic_utils import get_custom_objects
get_custom_objects().update({'modified_tanh': Activation(activation_fn)})

In [5]:
input_layer = tf.keras.layers.Input((32, 3))
lstm_1 = tf.keras.layers.LSTM(16, activation='modified_tanh', recurrent_activation='sigmoid', dropout = 0.15, return_sequences = True, return_state = True)(input_layer)
lstm_2 = tf.keras.layers.LSTM(64, activation='modified_tanh', recurrent_activation='sigmoid', dropout = 0.15, return_sequences = True, return_state = True)(lstm_1[0])
lstm_3 = tf.keras.layers.LSTM(256, activation='modified_tanh', recurrent_activation='sigmoid', dropout = 0.15, return_sequences = True, return_state = True)(lstm_2[0])
lstm_4 = tf.keras.layers.LSTM(512, activation='modified_tanh', recurrent_activation='sigmoid', dropout = 0.15, return_sequences = True, return_state = True)(lstm_3[0])
lstm_5 = tf.keras.layers.LSTM(256, activation='modified_tanh', recurrent_activation='sigmoid', dropout = 0.15, return_sequences = True, return_state = True)(lstm_4[0])
lstm_6 = tf.keras.layers.LSTM(64, activation='modified_tanh', recurrent_activation='sigmoid', dropout = 0.15, return_sequences = True, return_state = True)(lstm_5[0])
lstm_7 = tf.keras.layers.LSTM(16, activation='modified_tanh', recurrent_activation='sigmoid', dropout = 0.15, return_sequences = True, return_state = True)(lstm_6[0])
lstm_8 = tf.keras.layers.LSTM(2, activation='modified_tanh', recurrent_activation='sigmoid', dropout = 0.15, return_sequences = True, return_state = True)(lstm_7[0])
print(lstm_8[0])

KerasTensor(type_spec=TensorSpec(shape=(None, 32, 2), dtype=tf.float32, name=None), name='lstm_7/transpose_1:0', description="created by layer 'lstm_7'")


In [6]:
model = tf.keras.models.Model(inputs = input_layer, outputs = lstm_8[0], name = "DLVS_Model")

In [7]:
model.summary()

Model: "DLVS_Model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 3)]           0         
_________________________________________________________________
lstm (LSTM)                  [(None, 32, 16), (None, 1 1280      
_________________________________________________________________
lstm_1 (LSTM)                [(None, 32, 64), (None, 6 20736     
_________________________________________________________________
lstm_2 (LSTM)                [(None, 32, 256), (None,  328704    
_________________________________________________________________
lstm_3 (LSTM)                [(None, 32, 512), (None,  1574912   
_________________________________________________________________
lstm_4 (LSTM)                [(None, 32, 256), (None,  787456    
_________________________________________________________________
lstm_5 (LSTM)                [(None, 32, 64), (None, 6 8

In [8]:
### LOSS ###

In [9]:
def loss(y_true, y_pred):
    y_true = tf.convert_to_tensor(y_true, dtype = tf.float32)
    y_pred = tf.convert_to_tensor(y_pred, dtype = tf.float32)
    l1 = tf.reduce_mean(tf.math.abs((y_true - y_pred)))
    l2 = tf.reduce_mean(tf.math.pow((y_true, y_pred), 2))
    loss = l1 + l2
    return loss

In [10]:
import os
error_final = []
vel_final = []
path = "/home/uas-dtu/Documents/DL_IBVS/data_dynamic/"
for a in os.listdir(path):
    file = open(path + "/" + a, "r")
    for i in [file.readlines()]:
        for j in range(len(i) - 32):
#             print(j)
            error_data = []
            vel_data = []
            for k in range(32):
                #print(i[j + k].split(",")[3].split('[')[1].split("]")[0])
                #rint(float(i[j + k].split(",")[5]))
                #print(i[j + k].split(",")[4].split("\n"))
                error_data.append([float(i[j + k].split(",")[0]), 
                                   float(i[j + k].split(",")[1]),
                                   float(i[j + k].split(",")[2])])
                vel_data.append([float(i[j + k].split(",")[3].split('[')[1].split("]")[0]),
                                 float(i[j + k].split(",")[4].split('[')[1].split("]")[0])])
                
                #print(error_data)
            error_final.append(error_data)
            vel_final.append(vel_data)

In [11]:
print(len(vel_final))

15890


In [12]:
import numpy as np
import cv2
from tensorflow.keras.utils import Sequence
import tensorflow as tf
import os
import json
import random


class DataGenerator(Sequence):
    def __init__(self, error_final, vel_final,
                 batch_size=32,
                 shuffle=True):
        self.error_final = error_final
        self.vel_final = vel_final
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.on_epoch_end()

    def on_epoch_end(self):
        if self.shuffle:
            temp = [i for i in range(len(self.error_final))]
            random.shuffle(temp)
            #       self.flow_videos = [self.flow_videos[i] for i in temp]
            self.error_final = [self.error_final[i] for i in temp]
            self.vel_final = [self.vel_final[i] for i in temp]

    def __len__(self):
        return int(np.floor(len(self.error_final) / self.batch_size))


    def __getitem__(self, index):
        start_index = index * self.batch_size
#         input_1 = []
        error_reading = []
        vel_reading = []
        i = start_index - 1
        while len(error_reading) < self.batch_size:
            try:
                i += 1
                error = self.error_final[i%len(self.error_final)]
                vel = self.vel_final[i%len(self.vel_final)]
                error_reading.append(error)
                vel_reading.append(vel)

            except Exception as err:
                print(err)
                continue
            
        vel_reading = np.array(vel_reading, dtype = np.float32)
        error_reading = np.array(error_reading, dtype = np.float32)
            
        return error_reading, vel_reading

In [13]:
train_data = DataGenerator(error_final, vel_final, batch_size=32, shuffle=True)

In [14]:
print(train_data.__getitem__(0)[0].shape)

(32, 32, 3)


In [15]:
from tensorflow.keras.callbacks import (
    ModelCheckpoint,
    TensorBoard
)
import tensorflow as tf

In [16]:
callbacks = [tf.keras.callbacks.ModelCheckpoint("/home/uas-dtu/Documents/DL_IBVS/dynamicStats/train_{epoch}.tf", verbose = 1,
                                                save_weights_only=True),
             TensorBoard("/home/uas-dtu/Documents/DL_IBVS/dynamicStats/combined_logs")]

In [17]:
def mse(y_true, y_pred):
    y_true = tf.convert_to_tensor(y_true, dtype = tf.float32)
    y_pred = tf.convert_to_tensor(y_pred, dtype=tf.float32)
    l1 = tf.reduce_mean(tf.math.pow((y_pred - y_true), 2))
    l2 = tf.reduce_mean(tf.math.abs((y_pred - y_true)))
    loss = l1 + l2
    return loss

In [18]:
from tensorflow.keras.optimizers import Adam
optimizer=Adam(learning_rate=0.0001)

In [19]:
model.compile(optimizer=optimizer,loss=mse, metrics = [tf.keras.metrics.RootMeanSquaredError()])

In [20]:
# model.load_weights("/home/uas-dtu/Documents/DL_IBVS/dynamic+static/train_11.tf")

In [ ]:
model.fit(train_data, epochs=2000, verbose=1, callbacks=callbacks, initial_epoch = 0)

Epoch 1/2000
496/496 [==============================] - 413s 806ms/step - loss: 1.3924 - root_mean_squared_error: 0.8455

Epoch 00001: saving model to /home/uas-dtu/Documents/DL_IBVS/dynamicStats/train_1.tf
Epoch 2/2000
496/496 [==============================] - 396s 798ms/step - loss: 1.2358 - root_mean_squared_error: 0.7833

Epoch 00002: saving model to /home/uas-dtu/Documents/DL_IBVS/dynamicStats/train_2.tf
Epoch 3/2000
496/496 [==============================] - 399s 804ms/step - loss: 1.1674 - root_mean_squared_error: 0.7552

Epoch 00003: saving model to /home/uas-dtu/Documents/DL_IBVS/dynamicStats/train_3.tf
Epoch 4/2000
496/496 [==============================] - 394s 794ms/step - loss: 1.1204 - root_mean_squared_error: 0.7358

Epoch 00004: saving model to /home/uas-dtu/Documents/DL_IBVS/dynamicStats/train_4.tf
Epoch 5/2000
496/496 [==============================] - 391s 789ms/step - loss: 1.0856 - root_mean_squared_error: 0.7216

Epoch 00005: saving model to /home/uas-dtu/Documen

496/496 [==============================] - 397s 800ms/step - loss: 0.6783 - root_mean_squared_error: 0.5437

Epoch 00040: saving model to /home/uas-dtu/Documents/DL_IBVS/dynamicStats/train_40.tf
Epoch 41/2000
496/496 [==============================] - 398s 802ms/step - loss: 0.6738 - root_mean_squared_error: 0.5417

Epoch 00041: saving model to /home/uas-dtu/Documents/DL_IBVS/dynamicStats/train_41.tf
Epoch 42/2000
496/496 [==============================] - 397s 801ms/step - loss: 0.6684 - root_mean_squared_error: 0.5392

Epoch 00042: saving model to /home/uas-dtu/Documents/DL_IBVS/dynamicStats/train_42.tf
Epoch 43/2000
496/496 [==============================] - 397s 800ms/step - loss: 0.6599 - root_mean_squared_error: 0.5351

Epoch 00043: saving model to /home/uas-dtu/Documents/DL_IBVS/dynamicStats/train_43.tf
Epoch 44/2000
496/496 [==============================] - 396s 797ms/step - loss: 0.6620 - root_mean_squared_error: 0.5368

Epoch 00044: saving model to /home/uas-dtu/Documents/DL

496/496 [==============================] - 399s 805ms/step - loss: 0.5379 - root_mean_squared_error: 0.4778

Epoch 00079: saving model to /home/uas-dtu/Documents/DL_IBVS/dynamicStats/train_79.tf
Epoch 80/2000
496/496 [==============================] - 397s 800ms/step - loss: 0.5333 - root_mean_squared_error: 0.4753

Epoch 00080: saving model to /home/uas-dtu/Documents/DL_IBVS/dynamicStats/train_80.tf
Epoch 81/2000
496/496 [==============================] - 398s 802ms/step - loss: 0.5299 - root_mean_squared_error: 0.4734

Epoch 00081: saving model to /home/uas-dtu/Documents/DL_IBVS/dynamicStats/train_81.tf
Epoch 82/2000
496/496 [==============================] - 397s 800ms/step - loss: 0.5292 - root_mean_squared_error: 0.4730

Epoch 00082: saving model to /home/uas-dtu/Documents/DL_IBVS/dynamicStats/train_82.tf
Epoch 83/2000
496/496 [==============================] - 394s 794ms/step - loss: 0.5263 - root_mean_squared_error: 0.4720

Epoch 00083: saving model to /home/uas-dtu/Documents/DL

496/496 [==============================] - 398s 802ms/step - loss: 0.4629 - root_mean_squared_error: 0.4403

Epoch 00118: saving model to /home/uas-dtu/Documents/DL_IBVS/dynamicStats/train_118.tf
Epoch 119/2000
496/496 [==============================] - 399s 805ms/step - loss: 0.4574 - root_mean_squared_error: 0.4377

Epoch 00119: saving model to /home/uas-dtu/Documents/DL_IBVS/dynamicStats/train_119.tf
Epoch 120/2000
496/496 [==============================] - 396s 798ms/step - loss: 0.4542 - root_mean_squared_error: 0.4354

Epoch 00120: saving model to /home/uas-dtu/Documents/DL_IBVS/dynamicStats/train_120.tf
Epoch 121/2000
496/496 [==============================] - 399s 804ms/step - loss: 0.4521 - root_mean_squared_error: 0.4341

Epoch 00121: saving model to /home/uas-dtu/Documents/DL_IBVS/dynamicStats/train_121.tf
Epoch 122/2000
496/496 [==============================] - 396s 798ms/step - loss: 0.4488 - root_mean_squared_error: 0.4328

Epoch 00122: saving model to /home/uas-dtu/Docu

496/496 [==============================] - 394s 795ms/step - loss: 0.4066 - root_mean_squared_error: 0.4105

Epoch 00157: saving model to /home/uas-dtu/Documents/DL_IBVS/dynamicStats/train_157.tf
Epoch 158/2000
496/496 [==============================] - 396s 799ms/step - loss: 0.4015 - root_mean_squared_error: 0.4073

Epoch 00158: saving model to /home/uas-dtu/Documents/DL_IBVS/dynamicStats/train_158.tf
Epoch 159/2000
496/496 [==============================] - 397s 800ms/step - loss: 0.3970 - root_mean_squared_error: 0.4046

Epoch 00159: saving model to /home/uas-dtu/Documents/DL_IBVS/dynamicStats/train_159.tf
Epoch 160/2000
496/496 [==============================] - 396s 799ms/step - loss: 0.4002 - root_mean_squared_error: 0.4067

Epoch 00160: saving model to /home/uas-dtu/Documents/DL_IBVS/dynamicStats/train_160.tf
Epoch 161/2000
496/496 [==============================] - 396s 798ms/step - loss: 0.4002 - root_mean_squared_error: 0.4068

Epoch 00161: saving model to /home/uas-dtu/Docu

...

496/496 [==============================] - 401s 808ms/step - loss: 0.2625 - root_mean_squared_error: 0.3245

Epoch 00391: saving model to /home/uas-dtu/Documents/DL_IBVS/dynamicStats/train_391.tf
Epoch 392/2000
496/496 [==============================] - 396s 799ms/step - loss: 0.2627 - root_mean_squared_error: 0.3245

Epoch 00392: saving model to /home/uas-dtu/Documents/DL_IBVS/dynamicStats/train_392.tf
Epoch 393/2000
496/496 [==============================] - 398s 803ms/step - loss: 0.2621 - root_mean_squared_error: 0.3239

Epoch 00393: saving model to /home/uas-dtu/Documents/DL_IBVS/dynamicStats/train_393.tf
Epoch 394/2000
496/496 [==============================] - 400s 806ms/step - loss: 0.2635 - root_mean_squared_error: 0.3250

Epoch 00394: saving model to /home/uas-dtu/Documents/DL_IBVS/dynamicStats/train_394.tf
Epoch 395/2000
496/496 [==============================] - 397s 801ms/step - loss: 0.2617 - root_mean_squared_error: 0.3237

Epoch 00395: saving model to /home/uas-dtu/Docu

496/496 [==============================] - 397s 800ms/step - loss: 0.2514 - root_mean_squared_error: 0.3176

Epoch 00430: saving model to /home/uas-dtu/Documents/DL_IBVS/dynamicStats/train_430.tf
Epoch 431/2000
496/496 [==============================] - 394s 794ms/step - loss: 0.2505 - root_mean_squared_error: 0.3167

Epoch 00431: saving model to /home/uas-dtu/Documents/DL_IBVS/dynamicStats/train_431.tf
Epoch 432/2000
496/496 [==============================] - 397s 800ms/step - loss: 0.2546 - root_mean_squared_error: 0.3201

Epoch 00432: saving model to /home/uas-dtu/Documents/DL_IBVS/dynamicStats/train_432.tf
Epoch 433/2000
496/496 [==============================] - 391s 788ms/step - loss: 0.2499 - root_mean_squared_error: 0.3160

Epoch 00433: saving model to /home/uas-dtu/Documents/DL_IBVS/dynamicStats/train_433.tf
Epoch 434/2000
496/496 [==============================] - 398s 802ms/step - loss: 0.2512 - root_mean_squared_error: 0.3171

Epoch 00434: saving model to /home/uas-dtu/Docu

496/496 [==============================] - 397s 801ms/step - loss: 0.2430 - root_mean_squared_error: 0.3114

Epoch 00469: saving model to /home/uas-dtu/Documents/DL_IBVS/dynamicStats/train_469.tf
Epoch 470/2000
496/496 [==============================] - 391s 789ms/step - loss: 0.2419 - root_mean_squared_error: 0.3106

Epoch 00470: saving model to /home/uas-dtu/Documents/DL_IBVS/dynamicStats/train_470.tf
Epoch 471/2000
496/496 [==============================] - 392s 791ms/step - loss: 0.2375 - root_mean_squared_error: 0.3068

Epoch 00471: saving model to /home/uas-dtu/Documents/DL_IBVS/dynamicStats/train_471.tf
Epoch 472/2000
496/496 [==============================] - 396s 799ms/step - loss: 0.2434 - root_mean_squared_error: 0.3122

Epoch 00472: saving model to /home/uas-dtu/Documents/DL_IBVS/dynamicStats/train_472.tf
Epoch 473/2000
496/496 [==============================] - 395s 796ms/step - loss: 0.2440 - root_mean_squared_error: 0.3128

Epoch 00473: saving model to /home/uas-dtu/Docu

496/496 [==============================] - 395s 797ms/step - loss: 0.2330 - root_mean_squared_error: 0.3046

Epoch 00508: saving model to /home/uas-dtu/Documents/DL_IBVS/dynamicStats/train_508.tf
Epoch 509/2000
496/496 [==============================] - 396s 798ms/step - loss: 0.2329 - root_mean_squared_error: 0.3047

Epoch 00509: saving model to /home/uas-dtu/Documents/DL_IBVS/dynamicStats/train_509.tf
Epoch 510/2000
496/496 [==============================] - 399s 803ms/step - loss: 0.2387 - root_mean_squared_error: 0.3094

Epoch 00510: saving model to /home/uas-dtu/Documents/DL_IBVS/dynamicStats/train_510.tf
Epoch 511/2000
496/496 [==============================] - 394s 794ms/step - loss: 0.2330 - root_mean_squared_error: 0.3046

Epoch 00511: saving model to /home/uas-dtu/Documents/DL_IBVS/dynamicStats/train_511.tf
Epoch 512/2000
496/496 [==============================] - 395s 795ms/step - loss: 0.2311 - root_mean_squared_error: 0.3034

Epoch 00512: saving model to /home/uas-dtu/Docu

496/496 [==============================] - 397s 800ms/step - loss: 0.2258 - root_mean_squared_error: 0.2995

Epoch 00547: saving model to /home/uas-dtu/Documents/DL_IBVS/dynamicStats/train_547.tf
Epoch 548/2000
496/496 [==============================] - 396s 798ms/step - loss: 0.2251 - root_mean_squared_error: 0.2989

Epoch 00548: saving model to /home/uas-dtu/Documents/DL_IBVS/dynamicStats/train_548.tf
Epoch 549/2000
496/496 [==============================] - 395s 797ms/step - loss: 0.2259 - root_mean_squared_error: 0.2994

Epoch 00549: saving model to /home/uas-dtu/Documents/DL_IBVS/dynamicStats/train_549.tf
Epoch 550/2000
496/496 [==============================] - 397s 801ms/step - loss: 0.2247 - root_mean_squared_error: 0.2986

Epoch 00550: saving model to /home/uas-dtu/Documents/DL_IBVS/dynamicStats/train_550.tf
Epoch 551/2000
278/496 [===============>..............] - ETA: 2:54 - loss: 0.2262 - root_mean_squared_error: 0.3002